In [ ]:
using RecipesBase, StatsBase

"""
    f(event_list::EventList, bin_size::Number=0.1)

Recipe for plotting a light curve from an EventList.

# Arguments
- `event_list`: The EventList containing time data
- `bin_size`: Bin size for the light curve (default: 0.1)

# Returns
- Plot data for a light curve
"""
@recipe function f(event_list::EventList, bin_size::Number=0.1)
    bin_size_T = convert(eltype(event_list.times), bin_size)
    lc = create_lightcurve(event_list, bin_size_T)
    
    title --> "Light Curve"
    xlabel --> "Time (s)"
    ylabel --> "Counts"
    seriestype --> :scatter
    markershape --> :circle
    markersize --> 3
    yerror --> lc.count_error
    
    lc.timebins, lc.counts
end

"""
    GTIBTIPlot{T}

A type representing GTI or BTI plots.

# Fields
- `event_list`: The EventList to plot
- `plot_type`: Symbol indicating the type of plot (:bti_hist or :gti_info)
- `bins`: Number of bins for histogram
- `min_value`: Minimum value for histogram
- `max_value`: Maximum value for histogram
"""
struct GTIBTIPlot{T}
    event_list::EventList{T}
    plot_type::Symbol
    bins::Int
    min_value::Union{Nothing,T}
    max_value::Union{Nothing,T}
end

"""
    bti_histogram(event_list::EventList{T}; bins=30, min_value=nothing, max_value=nothing) where T

Create a BTI histogram plot.

# Arguments
- `event_list`: The EventList containing time data
- `bins`: Number of bins for the histogram
- `min_value`: Minimum value for the histogram
- `max_value`: Maximum value for the histogram

# Returns
- GTIBTIPlot object for plotting
"""
function bti_histogram(event_list::EventList{T}; 
                      bins=30, 
                      min_value=nothing, 
                      max_value=nothing) where T
    return GTIBTIPlot{T}(event_list, :bti_hist, bins, min_value, max_value)
end

"""
    gti_info(event_list::EventList{T}) where T

Create a GTI info plot.

# Arguments
- `event_list`: The EventList containing time data

# Returns
- GTIBTIPlot object for plotting
"""
function gti_info(event_list::EventList{T}) where T
    return GTIBTIPlot{T}(event_list, :gti_info, 0, nothing, nothing)
end

"""
    create_gti_from_eventlist(event_list::EventList)

Helper function to create GTI from event list.

# Arguments
- `event_list`: The EventList containing time data

# Returns
- Matrix representing Good Time Intervals
"""
function create_gti_from_eventlist(event_list::EventList)
    if isempty(event_list.times)
        error("Event list has no time data")
    end
    
    min_time = minimum(event_list.times)
    max_time = maximum(event_list.times)
    
    buffer = convert(eltype(event_list.times), 0.001)
    
    return reshape([min_time - buffer, max_time + buffer], 1, 2)
end

"""
    f(gp::GTIBTIPlot)

Recipe for plotting GTI or BTI information.

# Arguments
- `gp`: GTIBTIPlot object containing plot configuration

# Returns
- Plot data based on the specified plot type
"""
@recipe function f(gp::GTIBTIPlot)
    event_list = gp.event_list
    plot_type = gp.plot_type
    
    gti = create_gti_from_eventlist(event_list)
    
    if plot_type == :bti_hist
        bins = gp.bins
        min_value = gp.min_value
        max_value = gp.max_value
        
        btis_data = [gti[1,1] gti[1,2]]
        
        bti_lengths_data = [btis_data[1,2] - btis_data[1,1]]
        
        title --> "Event Duration Distribution"
        xlabel --> "Duration (s)"
        ylabel --> "Counts"
        seriestype --> :bar
        
        min_val = isnothing(min_value) ? 0.9 * bti_lengths_data[1] : min_value
        max_val = isnothing(max_value) ? 1.1 * bti_lengths_data[1] : max_value
        
        bin_edges = range(min_val, max_val, length=bins+1)
        
        counts = zeros(Int, bins)
        bin_idx = searchsortedfirst(bin_edges, bti_lengths_data[1]) - 1
        if 1 <= bin_idx <= bins
            counts[bin_idx] = 1
        end
        
        bin_centers = [(bin_edges[i] + bin_edges[i+1])/2 for i in 1:bins]
        return bin_centers, counts
        
    elseif plot_type == :gti_info
        total_duration = gti[1,2] - gti[1,1]
        
        title --> "Event Information"
        legend --> false
        grid --> false
        showaxis --> false
        ticks --> false
        seriestype := :annotate
        
        return [(0.5, 0.7, text("Total duration: $(round(total_duration, digits=2)) s", 12)),
                (0.5, 0.5, text("Number of events: $(length(event_list.times))", 12)),
                (0.5, 0.3, text("Event rate: $(round(length(event_list.times)/total_duration, digits=2)) events/s", 12))]
    end
end